In [1]:
import math
import numpy as np
import scipy.interpolate

# Características de asa finita:

Essas são as características de uma asa não ideal. Elas são obtidas através do perfil.

Aqui as características do perfil serão vistas como já definidas.
Em especial, temos que destacar o coeficiente angular do perfil $a_0$ 
$$ a_0 = \frac{\Delta c_l}{\Delta \alpha} $$



## Asas de alto alongamento (AR>4)

Se AR>4:

$\delta(\lambda,AR) $

In [2]:
def build_delta_lut(file, skip_header=True):
    lut = np.genfromtxt(file, delimiter='\t', dtype=float, skip_header=skip_header) #TO DO: use memoization?
    return lut

delta_lut = build_delta_lut('deltaLUT.tsv')
delta_lut[0:5]

array([[0.0815, 0.0946, 0.1077, 0.1165, 0.1252, 0.1414, 0.1575],
       [0.0767, 0.0889, 0.1011, 0.1093, 0.1174, 0.1344, 0.1513],
       [0.0705, 0.0825, 0.0944, 0.1019, 0.1093, 0.1274, 0.1455],
       [0.0627, 0.0761, 0.0894, 0.0961, 0.1028, 0.1215, 0.1401],
       [0.0556, 0.0692, 0.0827, 0.09  , 0.0973, 0.1161, 0.1348]])

In [3]:
def build_delta_interpolated_fuction(delta_lut):
    trs = np.arange(0, 1+0.005, 0.005)
    ars = np.arange(4,10+1, 1)

    delta = scipy.interpolate.interp2d(ars, trs, delta_lut, kind='linear')
    return delta

delta_interpolated_fuction = build_delta_interpolated_fuction(delta_lut)

def delta(tr, AR, delta_interpolated_fuction):
    if tr < 0 or tr > 1:
        raise ValueError("tr must be between 0 and 1")
    if AR < 4 or AR > 10:
        raise ValueError("AR must be between 4 and 10")
    return float(delta_interpolated_fuction(tr, AR))

delta(0, 4, delta_interpolated_fuction)

0.037

In [4]:
def delta(tr, AR, delta_lut):
    trs = np.arange(0, 1+0.005, 0.005)
    ars = np.arange(4,10+1, 1)
    return 



trs = np.arange(0, 1+0.005, 0.005)
ars = np.arange(4,10+1, 1)

delta = scipy.interpolate.interp2d(ars, trs, delta_lut, kind='linear')

$e = \dfrac {1} {1+ \delta}$

In [5]:
def e(tr, AR):
    return 1/(1+ delta(tr, AR) )

$a = \dfrac{a_0}{1+ (a_0 / \pi \cdot e \cdot AR)}$

In [6]:
def high_AR_a(AR, a_0, tr):
    return a_0 /\
        (1 +\
            (
                a_0/(math.pi * e(tr,AR) * AR)
            )
        )

Para asas de baixo alongamento:
Se AR ≤ 4


$a=\dfrac{a_0}{{1+\left(\dfrac{a_0}{\pi \cdot AR}\right)^2} + \dfrac{a_0}{\pi \cdot AR}}$

In [7]:
def low_AR_a(AR, a_0):
    return a_0 \
        /(1 + \
            (a_0/(math.pi * AR)**2 + \
            a_0/(math.pi * AR) \
            )
        )

In [8]:
def a(AR, a_0, tr):
    if AR>4:
        return high_AR_a(AR, a_0, tr)
    if AR <= 4 and AR > 0:
        return low_AR_a(AR, a_0)

a(5, 3, 1)

array([2.50406407])